In [1]:
# Uncomment the line below to install on Colab or similar
#! pip install git+https://github.com/monash-emu/renewal.git

In [2]:
from jax import jit, random
import pandas as pd
from datetime import datetime, timedelta
import numpyro
from numpyro import distributions as dist
from numpyro import infer
import arviz as az
from plotly.express.colors import qualitative as qual_colours
import numpy as np

from estival.sampling import tools as esamp

from emu_renewal.process import CosineMultiCurve
from emu_renewal.distributions import GammaDens
from emu_renewal.renew import RenewalModel, ModelResult
from emu_renewal.outputs import get_spaghetti_from_params, get_quant_df_from_spaghetti, new_get_spaghetti, new_new_get_spaghetti
from emu_renewal.outputs import plot_uncertainty_patches, plot_post_prior_comparison
from emu_renewal.calibration import StandardCalib
from emu_renewal.targets import StandardTarget


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/Users/jamestrauer/miniconda3/envs/renewal/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/Users/jamestrauer/miniconda3/envs/renewal/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/Users/jamestrauer/miniconda3/envs/renewal/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/Users/jamestrauer/miniconda3/envs/renewal/lib/python3.10/site-packages/traitlets

AttributeError: _ARRAY_API not found

In [3]:
# Specify fixed parameters and get calibration data
proc_update_freq = 14
window_len = 32
mys_data = pd.read_csv("https://github.com/monash-emu/wpro_working/raw/main/data/new_cases.csv", index_col=0)["MYS"]
mys_data.index = pd.to_datetime(mys_data.index)
pop = 33e6
analysis_start = datetime(2021, 3, 1)
analysis_end = datetime(2021, 11, 1)
init_start = analysis_start - timedelta(window_len)
init_end = analysis_start - timedelta(1)
select_data = mys_data.loc[analysis_start: analysis_end]
init_data = mys_data.loc[init_start: init_end]

In [4]:
renew_model = RenewalModel(33e6, analysis_start, analysis_end, proc_update_freq, CosineMultiCurve(), GammaDens(), window_len, init_data, GammaDens())

XlaRuntimeError: INVALID_ARGUMENT: Unknown NumPy dtype dtype('float64') char d kind f itemsize 0

In [ ]:
# Define parameter ranges
priors = {
    "gen_mean": dist.Uniform(6.5, 10.5),
    "gen_sd": dist.Uniform(3.0, 4.6),
    "cdr": dist.Beta(4.0, 10.0),
    "rt_init": dist.Normal(0.0, 0.25),
    "report_mean": dist.Uniform(8.0, 12.0),
    "report_sd": dist.Uniform(3.0, 6.0),
}

In [ ]:
targets = {
    "cases": StandardTarget(select_data, 0.1)
}

In [ ]:
calib = StandardCalib(renew_model, priors, targets)
kernel = infer.NUTS(calib.calibration, dense_mass=True, init_strategy=calib.custom_init(radius=0.5))
mcmc = infer.MCMC(kernel, num_chains=2, num_samples=100, num_warmup=100)
mcmc.run(random.PRNGKey(1))

In [ ]:
idata = az.from_dict(mcmc.get_samples(True))
idata_sampled = az.extract(idata, num_samples=200)
sample_params = esamp.xarray_to_sampleiterator(idata_sampled)

In [ ]:
spaghetti = new_new_get_spaghetti(calib, sample_params)
key_outputs = ["cases", "suscept", "r_t", "process"]
quantiles = get_quant_df_from_spaghetti(spaghetti, quantiles=[0.05, 0.5, 0.95])

In [ ]:
spaghetti

In [ ]:
quantiles=[0.5]
outputs = set(spaghetti.columns.get_level_values(0))
column_names = pd.MultiIndex.from_product([outputs, quantiles])
quantiles_df = pd.DataFrame(index=spaghetti.index, columns=column_names)
for col in outputs:
    quantiles_df[col] = spaghetti[col].quantile(quantiles, axis=1).T

In [ ]:
plot_uncertainty_patches(quantiles, select_data, qual_colours.Plotly, outputs=key_outputs).update_layout(showlegend=False)

In [ ]:
az.summary(idata)

In [ ]:
plot_post_prior_comparison(idata, list(priors.keys()), priors);

In [ ]:
parameters = {i: j for i, j in [k for k in sample_params.iterrows()][0][1].items() if "dispersion" not in i}
result = renew_model.renewal_func(**parameters)

In [ ]:
pd.DataFrame(
    {
        "cases": np.array(result.cases),
        "weekly_case": np.array(result.weekly_sum),
        "cases_target": np.array(targets["cases"].data)
    }
).plot()